In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

def preprocess_air_quality_data(file_path):
    # Load data
    df = pd.read_csv(file_path, low_memory=False)
    df.drop(columns=['Xylene','City'], inplace=True)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    # Process datetime and create time features
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed', errors='coerce')
    df['Year'] = df['Datetime'].dt.year
    df['Month'] = df['Datetime'].dt.month
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    
    # Categorize PM2.5 values
    bins = [0, 12, 35.4, 55.4, 150.4, 250.4, float('inf')]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['PM2.5_Category'] = pd.cut(df['PM2.5'], bins=bins, labels=labels)
    
    # Perform undersampling
    return df


# For each dataset
df = preprocess_air_quality_data('../Data/Bengaluru_data.csv')

# Print sizes to see the reduction
print("Original sizes:", len(df))

Original sizes: 48192


In [10]:
# df=pd.read_csv('../Data/Bengaluru_data.csv', low_memory=False)
df.head()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,AQI,AQI_Bucket,Year,Month,Day,Hour,PM2.5_Category
0,2015-01-01 01:00:00,35.578011,82.527733,2.04,12.40,7.58,19.10,0.19,4.05,12.41,0.560000,3.43,94.407038,NaN,2015,1,1,1,Unhealthy for Sensitive
1,2015-01-01 02:00:00,35.578011,82.527733,2.20,12.11,7.54,17.81,0.21,4.23,12.13,2.345345,4.54,94.407038,NaN,2015,1,1,2,Unhealthy for Sensitive
2,2015-01-01 03:00:00,35.578011,82.527733,1.66,10.82,6.52,17.42,0.30,4.28,13.13,2.345345,4.20,94.407038,NaN,2015,1,1,3,Unhealthy for Sensitive
3,2015-01-01 04:00:00,35.578011,82.527733,1.92,10.32,6.46,16.86,0.13,4.14,11.82,2.345345,4.63,94.407038,NaN,2015,1,1,4,Unhealthy for Sensitive
4,2015-01-01 05:00:00,35.578011,82.527733,1.94,9.82,6.22,16.35,0.15,4.26,10.31,2.345345,3.91,94.407038,NaN,2015,1,1,5,Unhealthy for Sensitive


In [11]:
df.describe()

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,AQI,Year,Month,Day,Hour
count,48192,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000,48192.000000
mean,2017-10-01 00:30:00,35.578011,82.527733,9.441072,28.057993,20.436661,21.599241,1.375805,5.477278,32.645345,2.345345,4.147976,94.407038,2017.271518,6.249626,15.723108,11.500000
min,2015-01-01 01:00:00,0.010000,0.010000,0.030000,0.060000,0.000000,0.020000,0.000000,0.010000,0.080000,0.000000,0.000000,10.000000,2015.000000,1.000000,1.000000,0.000000
25%,2016-05-17 00:45:00,19.640000,57.060000,3.450000,16.010000,8.177500,14.030000,0.610000,3.320000,14.800000,0.430000,0.980000,65.000000,2016.000000,3.000000,8.000000,5.750000
50%,2017-10-01 00:30:00,31.170000,82.527733,5.950000,24.560000,17.780000,21.390000,0.860000,4.810000,28.880000,0.970000,1.880000,85.000000,2017.000000,6.000000,16.000000,11.500000
75%,2019-02-15 00:15:00,41.690000,94.340000,10.740000,34.320000,27.532500,24.200000,1.250000,6.290000,41.500000,2.345345,4.147976,108.000000,2019.000000,9.000000,23.000000,17.250000
max,2020-07-01 00:00:00,999.990000,999.990000,287.040000,362.120000,326.400000,485.820000,49.960000,198.300000,199.940000,372.790000,499.400000,488.000000,2020.000000,12.000000,31.000000,23.000000
std,NaN,32.677987,42.773789,12.419454,19.114688,19.228925,14.800711,3.352820,6.387522,24.603738,9.743248,13.055711,47.900625,1.600125,3.439819,8.796540,6.922258


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48192 entries, 0 to 48191
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Datetime        48192 non-null  datetime64[ns]
 1   PM2.5           48192 non-null  float64       
 2   PM10            48192 non-null  float64       
 3   NO              48192 non-null  float64       
 4   NO2             48192 non-null  float64       
 5   NOx             48192 non-null  float64       
 6   NH3             48192 non-null  float64       
 7   CO              48192 non-null  float64       
 8   SO2             48192 non-null  float64       
 9   O3              48192 non-null  float64       
 10  Benzene         48192 non-null  float64       
 11  Toluene         48192 non-null  float64       
 12  AQI             48192 non-null  float64       
 13  AQI_Bucket      45497 non-null  object        
 14  Year            48192 non-null  int32         
 15  Mo

In [13]:
df.isna().sum()

Datetime             0
PM2.5                0
PM10                 0
NO                   0
NO2                  0
NOx                  0
NH3                  0
CO                   0
SO2                  0
O3                   0
Benzene              0
Toluene              0
AQI                  0
AQI_Bucket        2695
Year                 0
Month                0
Day                  0
Hour                 0
PM2.5_Category       0
dtype: int64

In [14]:
df_tmp = df.copy()

In [15]:
df_tmp.drop(['AQI_Bucket', 'Datetime'], axis=1, inplace=True)
df_tmp.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,AQI,Year,Month,Day,Hour,PM2.5_Category
0,35.578011,82.527733,2.04,12.40,7.58,19.10,0.19,4.05,12.41,0.560000,3.43,94.407038,2015,1,1,1,Unhealthy for Sensitive
1,35.578011,82.527733,2.20,12.11,7.54,17.81,0.21,4.23,12.13,2.345345,4.54,94.407038,2015,1,1,2,Unhealthy for Sensitive
2,35.578011,82.527733,1.66,10.82,6.52,17.42,0.30,4.28,13.13,2.345345,4.20,94.407038,2015,1,1,3,Unhealthy for Sensitive
3,35.578011,82.527733,1.92,10.32,6.46,16.86,0.13,4.14,11.82,2.345345,4.63,94.407038,2015,1,1,4,Unhealthy for Sensitive
4,35.578011,82.527733,1.94,9.82,6.22,16.35,0.15,4.26,10.31,2.345345,3.91,94.407038,2015,1,1,5,Unhealthy for Sensitive


In [16]:
df_tmp['PM2.5_Category'] = df_tmp['PM2.5_Category'].astype("category").cat.as_ordered()

In [17]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48192 entries, 0 to 48191
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   PM2.5           48192 non-null  float64 
 1   PM10            48192 non-null  float64 
 2   NO              48192 non-null  float64 
 3   NO2             48192 non-null  float64 
 4   NOx             48192 non-null  float64 
 5   NH3             48192 non-null  float64 
 6   CO              48192 non-null  float64 
 7   SO2             48192 non-null  float64 
 8   O3              48192 non-null  float64 
 9   Benzene         48192 non-null  float64 
 10  Toluene         48192 non-null  float64 
 11  AQI             48192 non-null  float64 
 12  Year            48192 non-null  int32   
 13  Month           48192 non-null  int32   
 14  Day             48192 non-null  int32   
 15  Hour            48192 non-null  int32   
 16  PM2.5_Category  48192 non-null  category
dtypes: category(

In [18]:
df_tmp['PM2.5_Category'].cat.categories

Index(['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy',
       'Very Unhealthy', 'Hazardous'],
      dtype='object')

In [19]:
df_tmp['PM2.5_Category'].cat.codes

0        2
1        2
2        2
3        2
4        2
        ..
48187    1
48188    1
48189    1
48190    1
48191    1
Length: 48192, dtype: int8

In [20]:
df_tmp.isnull().sum()/len(df_tmp)

PM2.5             0.0
PM10              0.0
NO                0.0
NO2               0.0
NOx               0.0
NH3               0.0
CO                0.0
SO2               0.0
O3                0.0
Benzene           0.0
Toluene           0.0
AQI               0.0
Year              0.0
Month             0.0
Day               0.0
Hour              0.0
PM2.5_Category    0.0
dtype: float64

In [21]:
np.random.seed(42)
from sklearn.model_selection import train_test_split

X = df_tmp.drop("PM2.5", axis=1)
Y = df_tmp["PM2.5"]

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8)

len(X_train), len(X_valid)

X_train.shape, X_valid.shape

((38553, 16), (9639, 16))

In [22]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)


PM10
NO
NO2
NOx
NH3
CO
SO2
O3
Benzene
Toluene
AQI
Year
Month
Day
Hour


In [23]:
for label, content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label, "--> has", pd.isnull(content).sum(), "null values")

In [24]:
pd.Categorical(X_train["PM2.5_Category"]).codes

array([2, 1, 3, ..., 3, 2, 2], dtype=int8)

In [25]:
for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #turn category into number
        X_train[label] = pd.Categorical(content).codes + 1

In [26]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38553 entries, 180 to 15795
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PM10            38553 non-null  float64
 1   NO              38553 non-null  float64
 2   NO2             38553 non-null  float64
 3   NOx             38553 non-null  float64
 4   NH3             38553 non-null  float64
 5   CO              38553 non-null  float64
 6   SO2             38553 non-null  float64
 7   O3              38553 non-null  float64
 8   Benzene         38553 non-null  float64
 9   Toluene         38553 non-null  float64
 10  AQI             38553 non-null  float64
 11  Year            38553 non-null  int32  
 12  Month           38553 non-null  int32  
 13  Day             38553 non-null  int32  
 14  Hour            38553 non-null  int32  
 15  PM2.5_Category  38553 non-null  int8   
dtypes: float64(11), int32(4), int8(1)
memory usage: 4.2 MB


In [27]:
for label, content in X_valid.items():
    if not pd.api.types.is_numeric_dtype(content):
        X_valid[label] = pd.Categorical(content).codes + 1

In [28]:
X_train.shape, X_valid.shape

((38553, 16), (9639, 16))

In [29]:
%%time
# Lets build a machine learning model
from sklearn.ensemble import RandomForestRegressor

CPU times: total: 0 ns
Wall time: 83.6 ms


In [30]:
np.random.seed(42)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    calculate root mean square log error between prediction and true labels    
    """

    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# create a function to evaluate model on a few different levels

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE" : mean_absolute_error(Y_train,train_preds),
              "valid MAE" : mean_absolute_error(Y_valid, val_preds),
              "Training RMSLE" : rmsle(Y_train,train_preds),
              "valid RMSLE" : rmsle(Y_valid, val_preds),
              "Training R^2": r2_score(Y_train,train_preds),
              "valid R^2" : r2_score(Y_valid, val_preds)}
    
    return scores

In [31]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [32]:
%%time
model.fit(X_train, Y_train)

CPU times: total: 1min 19s
Wall time: 7.73 s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [33]:
show_scores(model)

{'Training MAE': 1.6941798954872151,
 'valid MAE': 4.501068833372626,
 'Training RMSLE': 0.07453590236590495,
 'valid RMSLE': 0.1821866874269942,
 'Training R^2': 0.980701391170014,
 'valid R^2': 0.8801114523611832}

In [34]:
%%time

# most ideal parameter 

ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=18,
                                    min_samples_split=3,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)  # random state so that our reult is reproducable

# FIT THE MODEL

ideal_model.fit(X_train, Y_train)

CPU times: total: 18.3 s
Wall time: 1.8 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=18,
                      min_samples_split=3, n_estimators=90, n_jobs=-1,
                      random_state=42)

In [35]:
show_scores(ideal_model)

{'Training MAE': 4.337625920307571,
 'valid MAE': 4.899895756036545,
 'Training RMSLE': 0.18320564011780788,
 'valid RMSLE': 0.20263827767749076,
 'Training R^2': 0.8877104121457193,
 'valid R^2': 0.8373704729084023}

# Linear Regression

In [36]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,Y_train)

LinearRegression()

In [37]:
y_pred = linreg.predict(X_valid)

In [38]:
from sklearn.model_selection import cross_val_score
print('Cross_val_score', cross_val_score(linreg, X_train, Y_train, cv=10, scoring="r2").mean())
print('r2_score', r2_score(Y_valid, y_pred))
print("valid MAE ", mean_absolute_error(Y_valid, y_pred))

Cross_val_score 0.5424549184465974
r2_score 0.5025147276659583
valid MAE  9.579272885594934
